In [1]:
import requests
import lxml.html as lh
import pandas as pd
import re
import sqlite3 as sql
from sqlite3 import Error
from sqlalchemy import create_engine

In [2]:
def get_fund_details(url,investment):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    tr_elements = doc.xpath('//tr')
    col = []
    i = 0

    for t in tr_elements[0]:
        i+=1
        name = t.text_content()
        col.append((name,[]))
    for j in range(1,len(tr_elements)):
        T = tr_elements[j]
        if len(T)!=7:
            break
        i = 0
        for t in T.iterchildren():
            data = t.text_content()
            if i>0:
                try:
                    data = int(data)
                except:
                    pass
            col[i][1].append(data)
            i+=1
    Dict = {title:column for (title,column) in col}
    df = pd.DataFrame(Dict)
    df = df.drop(axis=0,index=15)
    link = doc.xpath('//tr/td/a[@href]/@href')
    link = link[0:len(df)]
    link = ['https://groww.in'+ x for x in link]
    df['Link'] = link
    df['Fund Type'] = investment
    
    return df 
    

In [3]:
def mutual_funds_scraper():
    URL_Dict = {
            'Equity_Mutual_Funds':'https://groww.in/mutual-funds/category/best-equity-mutual-funds',
            'Liquid_Mutual_Funds':'https://groww.in/mutual-funds/category/best-liquid-mutual-funds',
            'Short_term_Mutual_Funds':'https://groww.in/mutual-funds/category/best-ultra-short-mutual-funds',
            'Arbitrage_Mutual_Funds':'https://groww.in/mutual-funds/category/best-arbitrage-mutual-funds',
            'Debt_Mutual_Funds':'https://groww.in/mutual-funds/category/best-debt-mutual-funds',
            'Dynamic_Mutual_Funds':'https://groww.in/mutual-funds/category/best-dynamic-mutual-funds'}
    df1 = pd.DataFrame()
    for url in URL_Dict:
        investment = url
        df1 = df1.append(get_fund_details(URL_Dict[url],investment),ignore_index=True)
    
    return df1

    

In [4]:
def connect_to_db(db_file):
    sql_conn = None
    try:
        sql_conn = sql.connect(db_file)
        return sql_conn
    except Error as err:
        print(err)
        
        if sql_conn is not None:
            sql_conn.close()    

In [5]:
def insert_value_to_table(table_name,data_frame):
    conn = connect_to_db("data.sqlite")
    if conn is not None:
        c = conn.cursor()
        c.execute('CREATE TABLE IF NOT EXISTS '+table_name+
                  '(id                  INTEGER,'
                  'listing_name         VARCHAR,'
                  'listing_category     VARCHAR,'
                  'listing_risk         VARCHAR,'
                  'listing_1yreturns    VARCHAR,'
                  'listing_rating       VARCHAR,'
                  'listing_fund_size    VARCHAR,'
                  'listing_link         VARCHAR,'
                  'listing_fund_type    VARCHAR)')
        df = data_frame
        df.insert(0,'ID',df.index)
        df = df.drop('',axis = 1)
        df.columns = get_column_names_from_db_table(c,table_name)
        df.to_sql(table_name,con=conn,if_exists='replace',index=False)
        conn.commit()
        conn.close()
        print('SQL insert process finished')
    else:
        print('Connection to database failed')

In [6]:
def get_column_names_from_db_table(sql_cursor,table_name):
    table_column_names = "PRAGMA table_info("+table_name+");"
    sql_cursor.execute(table_column_names)
    table_column_names = sql_cursor.fetchall()
    
    column_names = list()
    for name in table_column_names:
        column_names.append(name[1])
    return column_names

In [17]:
if __name__=="__main__":
    final = mutual_funds_scraper()
    insert_value_to_table('investments_list',final)

SQL insert process finished


In [ ]:
conn = sql.connect("fund.db")
cursor = conn.execute('''SELECT * FROM Table3''')
for row in cursor:
    print(row)

In [14]:
final['ID'] = final.index + 1

In [16]:
final = final.drop('Id',axis = 1)